In [1]:
import numpy as np
import pandas as pd
import plotly_express as px
import matplotlib.pyplot as plt
from Invoice import Invoice
from Enterprise import Enterprise

In [57]:
enterprise_info=pd.read_csv('./C/2_info_random.csv')
N_enterprise=enterprise_info.shape[0]

In [58]:
enterprise_dic={}
for i in range(N_enterprise):
    number = enterprise_info['企业代号'][i]
    name = enterprise_info['企业名称'][i]
    credit_rating = enterprise_info['信誉评级'][i]
    break_contract_str = enterprise_info['是否违约'][i]
    break_contract = True if break_contract_str=='是' else False
    enterprise_object=Enterprise(number,name,credit_rating,break_contract)
    enterprise_dic[number]=enterprise_object

In [59]:
anum = 0
bnum = 0
for i in range(N_enterprise):
    a = np.random.randint(1,6)
    if a == 1:
        anum = anum + 1
    else:
        bnum = bnum + 1
    enterprise_info['是否违约'][i] = '是' if a == 1 else '否'
enterprise_info

,企业代号,企业名称,信誉评级,是否违约
0,E124,个体经营E124,C,否
1,E125,个体经营E125,B,否
2,E126,个体经营E126,A,否
3,E127,个体经营E127,C,否
4,E128,个体经营E128,B,否
...,...,...,...,...
297,E421,***保温材料有限公司,A,是
298,E422,***童装店,B,否
299,E423,***通风设备有限公司,B,是
300,E424,***贸易有限公司,D,是


In [60]:
#enterprise_info.to_csv("./C/2_info_random.csv",index=False,sep=',',encoding='utf_8_sig')

In [61]:
invoice_in = pd.read_csv('./C/2_in.csv')
N_in = invoice_in.shape[0]

In [62]:
from datetime import date
def get_date(date_str):
    y,m,d=date_str.split('/')
    if len(m) != 2:
        m = '0' + m
    if len(d) != 2:
        d = '0' + d
    return date.fromisoformat(y+'-'+m+'-'+d)

In [63]:
for i in range(N_in):
    enterprise_name = invoice_in['企业代号'][i]
    number = invoice_in['发票号码'][i]
    date_str = invoice_in['开票日期'][i]
    date = get_date(date_str)
    self_enterprise = enterprise_dic[enterprise_name]
    partner = invoice_in['销方单位代号'][i]
    amount = invoice_in['金额'][i]
    tax = invoice_in['税额'][i]
    sum_money = invoice_in['价税合计'][i]
    state_available_str = invoice_in['发票状态'][i]
    state_available = True if state_available_str == '有效发票' else False
    invoice_object = Invoice(number,date,self_enterprise,partner,amount,tax,sum_money,state_available,True)
    self_enterprise.add_invoice(invoice_object)

KeyboardInterrupt: 

In [ ]:
invoice_out = pd.read_csv('./C/2_out.csv')
N_out = invoice_out.shape[0]

In [ ]:
for i in range(N_out):
    enterprise_name = invoice_out['企业代号'][i]
    number = invoice_out['发票号码'][i]
    date_str = invoice_out['开票日期'][i]
    date = get_date(date_str)
    self_enterprise = enterprise_dic[enterprise_name]
    partner = invoice_out['购方单位代号'][i]
    amount = invoice_out['金额'][i]
    tax = invoice_out['税额'][i]
    sum_money = invoice_out['价税合计'][i]
    state_available_str = invoice_out['发票状态'][i]
    state_available = True if state_available_str == '有效发票' else False
    invoice_object = Invoice(number,date,self_enterprise,partner,amount,tax,sum_money,state_available,False)
    self_enterprise.add_invoice(invoice_object)

In [ ]:
for enterprise in enterprise_dic.values():
    enterprise.invoice_list.sort(key=lambda x:x.date)

In [ ]:
frame = pd.DataFrame(columns=['日期','资金','企业代号'])
for enterprise in enterprise_dic.values():
    sum = 0
    current_date = enterprise.invoice_list[0].date
    for invoice in enterprise.invoice_list:
        if invoice.state_avaliable == False:
            continue
        temp = invoice
        if invoice.date!=current_date:
            frame = frame.append([{'日期':current_date,'资金':sum,'企业代号':enterprise.number}],ignore_index=True)
            current_date = invoice.date
        if invoice.buy_in == True:
            sum = sum - invoice.sum_money
        else:
            sum = sum + invoice.amount
    frame = frame.append([{'日期':temp.date,'资金':sum,'企业代号':enterprise.number}],ignore_index=True)
frame.to_csv("./C/2_收益波动_日期.csv",index=False,sep=',',encoding='utf_8_sig')

In [ ]:
graph = px.line(frame, x="日期", y="资金",color='企业代号',  render_mode="auto")
graph.write_html('./2_graph/total.html')

In [ ]:
frame1 = pd.DataFrame(columns=['日期','资金','企业代号','信誉评级'])
for enterprise in enterprise_dic.values():
    temp = frame[frame.企业代号 == enterprise.number].reset_index(drop=True)
    first_date = temp['日期'][0]
    for i in range(temp.shape[0]):
        frame1 = frame1.append([{'日期':(temp['日期'][i]-first_date).days,'资金':temp['资金'][i],'企业代号':temp['企业代号'][i]}],ignore_index=True)

In [ ]:
#frame1.to_csv("./C/2_收益波动.csv",index=False,sep=',',encoding='utf_8_sig')
frame1 = pd.read_csv("./C/2_收益波动.csv")

In [ ]:
k_mean_var_dic = {}
for enterprise in enterprise_dic.values():
    temp = frame1[frame1.企业代号 == enterprise.number].reset_index(drop=True)
    N = temp.shape[0]
    days = temp['日期'][N-1]
    ks=[]
    i=0
    while temp['日期'][i] + 30 < days:
        today = temp['日期'][i]
        min_index = i
        min_days = 1000
        for j in range(i+1,N):
            delta = temp['日期'][j] - temp['日期'][i] - 30
            if abs(delta) < min_days:
                min_days = abs(delta)
                min_index = j
            if delta > 0:
                break
        ks.append((temp['资金'][min_index] - temp['资金'][i])/(temp['日期'][min_index] - temp['日期'][i]))
        i=min_index
    k_mean_var_dic[enerprise.number] = [np.mean(ks),np.var(ks)]
frame7 = pd.DataFrame.from_dict(k_mean_var_dic,orient='index',columns=['增长率均值','增长率方差'])
frame7 = frame7.reset_index().retname(columns={'index':'企业代号'})
frame7.to_csv("./C/2_资金增长率均值方差.csv",index=False,sep=',',encoding='utf_8_sig')

In [ ]:
As = {}
for enterprise in enterprise_dic.values():
    temp = frame1[frame1.企业代号 == enterprise.number].reset_index(drop=True)
    N = temp.shape[0]
    days = temp['日期'][N-1]
    min_days = 365
    min_index = 0
    for i in range(N):
        if abs(days-temp['日期'][i]-365)<min_days:
            min_days=abs(days-temp['日期'][i]-365)
            min_index = i
    '''
    print('min_days:',min_days)
    print('min_index:',min_index)
    '''
    delta = temp['资金'][N-1] - temp['资金'][min_index]
    As[enterprise.number] = int(delta/3)
#As

In [ ]:
for en_num in As.keys():
    if As[en_num]>1000000:
        As[en_num]=1000000
    if As[en_num]<0:
        As[en_num]=0

In [ ]:
data_rate_and_loss = pd.read_csv("./C/rate_and_loss.csv")
def calc_alpha(r, credit_rating):
    for i in range(data_rate_and_loss.shape[0]):
        if data_rate_and_loss["贷款年利率"][i] == r:
            return data_rate_and_loss[credit_rating][i]

def calc_beta(A, F):
    return F(A)

def calc_gamma(credit_rating, break_contract):
    c = {"A": 1, "B": 0.9, "C": 0.7}
    b = {True: 0.5, False: 1}
    return c[credit_rating] * b[break_contract]

def calc_A_(A, F, f,enterprise: "Enterprise"):
    #v, err = integrate.quad(F, -np.inf, A)
    lower = frame4[frame4.企业代号 == enterprise.number].reset_index(drop=True)['资金'][0]
    #print(lower,F(lower))
    v = F.integ()(A)-F.integ()(lower)
    return (A * F(A)- lower * F(lower) - v) / (F(A)-F(lower))

def calc_expect_profit(A, r, enterprise: "Enterprise",F):
    alpha = calc_alpha(r, enterprise.credit_rating) # 客户流失率
    beta = calc_beta(A,F) # 资金抵债率
    gamma = calc_gamma(enterprise.credit_rating, enterprise.break_contract) # 企业信誉系数
    f=F.deriv()
    A_ = calc_A_(A,F,f,enterprise) # 无法还债时的期望剩余资金
    '''
    print('beta: ',beta)
    print('gamma: ',gamma)
    print('A_: ',A_)
    print('(A_ / gamma - A * (1 + r)) :',(A_ / gamma - A * (1 + r)))
    print('(A * r): ',(A * r))
    '''
    return (beta * (A_ / gamma - A) + (1 - beta) * (A * r)) * (1 - alpha)

In [ ]:
from scipy.stats import norm
from Enterprise import Enterprise

def calc(enterprise, A, r):
    temp = frame1[frame1.企业代号 == enterprise.number].reset_index(drop=True)
    #remain = predict(temp)
    #beta = 1.0 * A / remain[0] * 0.5
    beta = np.random.randint(5,15)/1000
    #if (remain[0] < 0):
    #    return -1

    gamma = calc_gamma(enterprise.credit_rating, enterprise.break_contract)
    #EL =  remain[0] / gamma - A
    EL = -A
    alpha = calc_alpha(r, enterprise.credit_rating)

    '''
    print(enterprise.number)
    print("remain=", remain[0])
    print("beta=", beta)
    print("gamma=", gamma)
    print("EL=", EL)
    print("alpha=", alpha)

    '''

    return 1.0 * (A * r * (1 - beta) + EL * beta) * (1 - alpha)

In [ ]:
rates = list(data_rate_and_loss["贷款年利率"])

In [ ]:
i=0
for enterprise in enterprise_dic.values():
    #enterprise = enterprise_dic['E71']
    #frame5 = pd.DataFrame(columns=['预期收益','贷款金额','年利率'])
    if enterprise.credit_rating == "D" or As[enterprise.number]==0:
        continue
    ans = 0
    ans_A = 0
    ans_r = 0
    A = As[enterprise.number]
    for r in rates:
        ans_ = calc(enterprise, A, r)
        #frame5 = frame5.append([{'预期收益':ans_,'贷款金额':A,'年利率':r}],ignore_index=True)
        # print(enterprise.number, ans_, A, r)
        if (ans_>ans):
            ans = ans_
            ans_A = A
            ans_r = r
    #print(enterprise.number,ans)
    print(enterprise.number, A, ans, ans_r)
    #graph4 = px.scatter_3d(frame5, x="贷款金额", y="年利率",z='预期收益')
    #graph4.write_html('./1_graph/预期收益/'+enterprise.number+'.html')
    

In [80]:
frame8 = pd.read_csv('answer2.csv')
#print(sum(frame8['额度']))
#graph = px.scatter(frame8, x="额度", y="年利率", size="预期收益", color="企业代号",hover_name="企业代号", log_x=True, size_max=30)
#graph.write_html('./answer2.html')

'''
frame8 = frame8.sort_values(by=['额度','企业代号'],axis=0,ascending=[False,True]).reset_index(drop=True)
graph2 = px.histogram(frame8, x="企业代号", y="额度",color_discrete_map=color_dic)
graph2.write_html('./2_graph/A倒序.html')
'''

'\nframe8 = frame8.sort_values(by=[\'额度\',\'企业代号\'],axis=0,ascending=[False,True]).reset_index(drop=True)\ngraph2 = px.histogram(frame8, x="企业代号", y="额度",color_discrete_map=color_dic)\ngraph2.write_html(\'./2_graph/A倒序.html\')\n'

In [54]:
color_dic = {}
for enterprise in enterprise_dic.keys():
    if frame10[frame10.企业代号 == enterprise].reset_index(drop=True)['均值×方差'][0] < 0 :
        color_dic[enterprise] = 'red'
    else:
        color_dic[enterprise] = 'blue'

In [2]:
frame9 = pd.read_csv('./C/2_资金增长率均值方差.csv')
frame10 = pd.DataFrame(columns=['企业代号','均值×方差'])
for i in range(frame9.shape[0]):
    frame10 = frame10.append([{'企业代号':frame9['企业代号'][i],'均值×方差':np.log2(frame9['增长率均值'][i]*frame9['增长率方差'][i]) if frame9['增长率均值'][i]*frame9['增长率方差'][i] > 0 else -np.log2(-frame9['增长率均值'][i]*frame9['增长率方差'][i])}],ignore_index=True)
frame10 = frame10.sort_values(by=['均值×方差','企业代号'],axis=0,ascending=[False,True]).reset_index(drop=True)
frame10.to_csv("./C/2_增长率均值×方差.csv",index=False,sep=',',encoding='utf_8_sig')
graph3 = px.histogram(frame10, x="企业代号", y="均值×方差",color_discrete_map=color_dic)
graph3.write_html('./2_graph/增长率均值×方差.html')

In [81]:
frame11 = pd.read_csv('./C/2_增长率均值×方差.csv')
frame8['增长率均值×方差']=''
for i in range(frame8.shape[0]):
    frame8['增长率均值×方差'][i] = frame11[frame11.企业代号 == frame8['企业代号'][i]].reset_index(drop=True)['均值×方差'][0]
#frame8 = frame8.sort_values(by=['增长率均值×方差','企业代号'],axis=0,ascending=[True,True]).reset_index(drop=True)
frame8 = frame8.sort_values(by=['额度','企业代号'],axis=0,ascending=[False,True]).reset_index(drop=True)
frame8

,企业代号,额度,预期收益,年利率,增长率均值×方差
0,E126,1000000,33857.455470,0.0465,54.6582
1,E127,1000000,35555.084077,0.0785,56.9879
2,E128,1000000,35616.035601,0.0585,54.2986
3,E129,1000000,34805.472013,0.0825,52.516
4,E131,1000000,37091.831441,0.0585,49.3263
...,...,...,...,...,...
186,E422,7617,269.235321,0.0625,19.2056
187,E393,5937,200.433120,0.0400,27.6263
188,E417,5933,219.615505,0.0585,21.7517
189,E402,5771,198.613551,0.0665,35.1919


In [163]:
frame8['调整后额度']=''
total = 19893799
for i in range(frame8.shape[0]):
    a = frame8['额度'][i]
    '''
    if a < 100000:
        frame8['调整后额度'][i] = a
        continue
    if total > 0:
        b = min(total,np.random.randint((a - 100000)/2,a - 100000))
        frame8['调整后额度'][i] = a - b
        total = total - b
    else:
        frame8['调整后额度'][i] = a
    '''
    if a < 100000:
        frame8['调整后额度'][i] = a
        continue
    if total > 0:
        #r = (a-100000)*0.02
        b = min(total,np.random.randint((a-100000)*0.05,(a-100000)*0.3789))
        frame8['调整后额度'][i] = a - b
        total = total - b
    else:
        frame8['调整后额度'][i] = a
frame8 = frame8.sort_values(by=['额度','企业代号'],axis=0,ascending=[False,True]).reset_index(drop=True)
#frame8.to_csv("./C/2_调整后额度_2.csv",index=False,sep=',',encoding='utf_8_sig')
frame8.to_csv("./C/3_调整后额度.csv",index=False,sep=',',encoding='utf_8_sig')

In [164]:
sum(frame8['调整后额度'])

100000000

In [165]:
#frame8 = frame8.sort_values(by=['额度','企业代号'],axis=0,ascending=[False,True]).reset_index(drop=True)
#frame8.to_csv("./C/2_调整后额度.csv",index=False,sep=',',encoding='utf_8_sig')
frame8 = pd.read_csv('./C/3_调整后额度.csv')

for i in range(frame8.shape[0]):
    if i%2 == 0:
        assert frame8['企业代号'][i] == frame8['企业代号'][i+1]
        frame8['额度'][i+1] = frame8['额度'][i+1]-frame8['额度'][i]
    else:
        continue
'''
for i in range(frame8.shape[0]):
    frame8['额度'][i] = frame8['额度'][i]-frame8['调整后额度'][i]'''

"\nfor i in range(frame8.shape[0]):\n    frame8['额度'][i] = frame8['额度'][i]-frame8['调整后额度'][i]"

In [166]:
graph3 = px.histogram(frame8, x="企业代号", y="额度",color='是否调整',color_discrete_map={'调整后':'green','调整前':'red'})
graph3.write_html('./answer3_adjust.html')
#frame9 = frame8.drop(['预期收益','年利率','增长率均值×方差'],axis=1)
#graph3 = px.bar(frame9, x="企业代号", y=["调整后额度","额度"],color_discrete_map={'额度':'red','调整后额度':'green'})
#graph3.write_html('./answer2_adjust_1.html')

In [66]:
frame8

,企业代号,额度,预期收益,年利率,增长率均值×方差,是否调整
0,E126,0,33857.455470,0.0465,54.658183,调整前
1,E126,1000000,33857.455470,0.0465,54.658183,调整后
2,E127,0,35555.084077,0.0785,56.987888,调整前
3,E127,1000000,35555.084077,0.0785,56.987888,调整后
4,E128,0,35616.035601,0.0585,54.298595,调整前
...,...,...,...,...,...,...
377,E417,5933,219.615505,0.0585,21.751691,调整后
378,E402,0,198.613551,0.0665,35.191931,调整前
379,E402,5771,198.613551,0.0665,35.191931,调整后
380,E412,0,98.639695,0.0585,24.510943,调整前


In [47]:
#frame8 = frame8.sort_values(by=['额度','企业代号'],axis=0,ascending=[False,True]).reset_index(drop=True)

for i in range(1,frame9.shape[0]):
    assert type(frame9['额度'][i-1]) == type(frame9['额度'][i])
    assert type(frame9['调整后额度'][i-1]) == type(frame9['调整后额度'][i])
    #frame9['额度'][i] = int(frame9['额度'][i])
    #frame9['调整后额度'][i] = int(frame9['调整后额度'][i])
frame9

,企业代号,额度,调整后额度
0,E126,0,1000000
1,E127,0,1000000
2,E128,0,1000000
3,E129,0,1000000
4,E131,0,1000000
...,...,...,...
186,E422,0,7617
187,E393,0,5937
188,E417,0,5933
189,E402,0,5771


In [60]:
wide_df = px.data.medals_wide()
fig = px.bar(wide_df, x="nation", y=["gold", "silver", "bronze"], title="Wide-Form Input")
fig.show()
wide_df


,nation,gold,silver,bronze
0,South Korea,24,13,11
1,China,10,15,8
2,Canada,9,12,12


In [62]:
frame10 = pd.DataFrame(columns=['企业代号','额度','调整后额度'])

for i in range(10):
    frame10 = frame10.append([{'企业代号':str(frame9['企业代号'][i]),'额度':frame9['额度'][i],'调整后额度':frame9['调整后额度'][i]}],ignore_index=True)
graph3 = px.bar(frame10, x="企业代号", y=["调整后额度","额度"])
graph3.write_html('./answer2_adjust_1.html')

In [55]:
frame10

,企业代号,额度,调整后额度
0,E126,0,1000000
1,E127,0,1000000
2,E128,0,1000000
3,E129,0,1000000
4,E131,0,1000000
...,...,...,...
186,E422,0,7617
187,E393,0,5937
188,E417,0,5933
189,E402,0,5771
